In [1]:
import os

os.chdir('/home/alfirsafauzulh@student.ub.ac.id/Channelwise-Saab-Transform')

In [2]:
# Saab transformationv 2021.04.12
# modified from https://github.com/ChengyaoWang/PixelHop-_c-wSaab/blob/master/saab.py

import numpy as np
import numba
# from sklearn.decomposition import PCA, IncrementalPCA


@numba.jit(nopython = True, parallel = True)
def pca_cal(X: np.ndarray):
    cov = X.transpose() @ X
    eva, eve = np.linalg.eigh(cov)
    inds = eva.argsort()[::-1]
    eva = eva[inds]
    kernels = eve.transpose()[inds]
    return kernels, eva / (X.shape[0] - 1)

@numba.jit(forceobj = True, parallel = True)
def remove_mean(X: np.ndarray, feature_mean: np.ndarray):
    return X - feature_mean

@numba.jit(nopython = True, parallel = True)
def feat_transform(X: np.ndarray, kernel: np.ndarray):
    return X @ kernel.transpose()


class Saab():
    def __init__(self, num_kernels=-1, needBias=True, bias=0):
        self.num_kernels = num_kernels 
        self.needBias = needBias
        self.Bias_previous = bias # bias calculated from previous
        self.Bias_current = [] # bias for the current Hop
        self.Kernels = []
        self.Mean0 = [] # feature mean of AC
        self.Energy = [] # kernel energy list
        self.trained = False

    def fit(self, X): 
        assert (len(X.shape) == 2), "Input must be a 2D array!"
        X = X.astype('float32')
        
        # add bias from the previous Hop
        if self.needBias == True:
            X += self.Bias_previous
            
        # remove DC, get AC components
        dc = np.mean(X, axis = 1, keepdims = True)
        X = remove_mean(X, dc)
        
        # calcualte bias at the current Hop
        self.Bias_current = np.max(np.linalg.norm(X, axis=1))
        
        # remove feature mean --> self.Mean0
        self.Mean0 = np.mean(X, axis = 0, keepdims = True)
        X = remove_mean(X, self.Mean0)

        if self.num_kernels == -1:
            self.num_kernels = X.shape[-1]
        
        # Rewritten PCA Using Numpy
        kernels, eva = pca_cal(X)
        
        # Concatenate with DC kernel
        dc_kernel = 1 / np.sqrt(X.shape[-1]) * np.ones((1, X.shape[-1]))# / np.sqrt(largest_ev)
        kernels = np.concatenate((dc_kernel, kernels[:-1]), axis = 0)
        
        # Concatenate with DC energy
        largest_ev = np.var(dc * np.sqrt(X.shape[-1]))  
        energy = np.concatenate((np.array([largest_ev]), eva[:-1]), axis = 0)
        energy = energy / np.sum(energy)
        
        # store
        self.Kernels, self.Energy = kernels.astype('float32'), energy
        self.trained = True


    def transform(self, X):
        assert (self.trained == True), "Must call fit first!"
        X = X.astype('float32')
        
        # add bias from the previous Hop
        if self.needBias == True:
            X += self.Bias_previous
            
        # remove feature mean of AC
        X = remove_mean(X, self.Mean0)
        
        # convolve with DC and AC filters
        X = feat_transform(X, self.Kernels)
        
        return X

In [3]:
# load all images from covid-chestxray-dataset/images

import os
import cv2

# list all files in covid-chestxray-dataset/images
path = 'covid-chestxray-dataset/images'
files = os.listdir(path)

In [4]:
img = cv2.imread(os.path.join(path, files[0]), cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (64,64))
img = img.reshape(1, 64,64, 1)

print(img)
print(" input feature shape: %s"%str(img.shape))

[[[[ 51]
   [ 50]
   [  0]
   ...
   [ 48]
   [ 48]
   [ 48]]

  [[ 50]
   [ 48]
   [115]
   ...
   [ 45]
   [ 47]
   [ 48]]

  [[ 49]
   [ 47]
   [ 47]
   ...
   [ 44]
   [ 46]
   [ 46]]

  ...

  [[175]
   [176]
   [179]
   ...
   [164]
   [155]
   [150]]

  [[173]
   [177]
   [176]
   ...
   [166]
   [155]
   [150]]

  [[175]
   [173]
   [176]
   ...
   [167]
   [157]
   [153]]]]
 input feature shape: (1, 64, 64, 1)


In [5]:
datasets = []
for i in range(len(files)):
    img = cv2.imread(os.path.join(path, files[i]), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (512,512))
    img = img.reshape(1, 512,512, 1)
    datasets.append(img)

datasets = np.concatenate(datasets, axis = 0)
print(" input feature shape: %s"%str(datasets.shape))

 input feature shape: (930, 512, 512, 1)


In [6]:
X = datasets.copy()
X = X.reshape(X.shape[0], -1)[0:5] 
print(" input feature shape: %s"%str(X.shape))

 input feature shape: (5, 262144)


In [7]:
saab = Saab(num_kernels=-1, needBias=True, bias=0)
saab.fit(X)

In [1]:
xt = saab.transform(X)
print(" output feature shape: %s"%str(xt.shape))


NameError: name 'saab' is not defined

In [12]:
print(" before xt ")
img = datasets[0].reshape(512, 512)
print(img)

print(" after xt ")
img = xt[0].reshape(512, 512)
print(img)


 before xt 
[[ 65  83 117 113 111  95  95  51]
 [106 112  74 120 107  76 112 115]
 [110  85  86 116 132  77  77 123]
 [111  76  74 125 132  88  70 120]
 [129  76  81 120 133  86  77 132]
 [137  78  83 134 116 129  85 115]
 [145 128 117 119 114 118 118 120]
 [135 112 113 111 112 109 111 117]]
 after xt 
[[ 8.45499878e+02 -8.75452118e+01  1.17194160e+02 -1.07152336e+02
  -8.97542286e+00  1.17920809e+01 -1.03148232e+01  1.95761013e+01]
 [ 9.62392426e+00  1.50919962e+01  2.21294994e+01 -3.80111885e+01
   2.76408052e+00  1.55926895e+01 -2.05647969e+00  5.52899456e+00]
 [-1.26376591e+01  1.42624140e+00 -8.01621246e+00 -2.79141960e+01
  -3.00571394e+00  9.92163086e+00  1.55805588e+01  4.04148054e+00]
 [ 9.16978550e+00 -3.65355849e+00 -4.34708643e+00 -5.11817455e+00
  -6.08921146e+00  5.17383337e-01 -1.04166069e+01  7.37631273e+00]
 [ 6.27021432e-01 -1.93904896e+01  8.65321064e+00  2.38075008e+01
   6.78848648e+00  2.32924476e-01  1.75064602e+01 -1.09700947e+01]
 [-1.79365520e+01 -5.22142458e+

In [12]:
# check ram usage
import psutil
print(psutil.virtual_memory())

svmem(total=8344264704, available=5898502144, percent=29.3, used=2121146368, free=318332928, active=3248369664, inactive=4274827264, buffers=184152064, cached=5720633344, shared=954368, slab=396378112)
